Estimativa de Função de Produção

Seja a função de produção Cobb-Douglas (log-linear):

$y_{jt} = \beta_k k_{jt} + \beta_l l_{jt} + \omega_{jt}$ (1)

tal que $x = ln(X)$, X é uma variável aleatória, y é o ln do produto (medido como valor adicionado) da firma j no período t, k é o ln do estoque de capital e $\omega$ a produtividade total dos fatores (PTF).
O problema básico da estimação dos parâmetros da função de produção diz que as variáveis k e l podem ser correlacionadas com o termo de erro, se estimada por OLS. Além de que k e l podem ser escolhidos juntamente com y.
Neste problema, a função de produção (1) deve ser estimada usando o método de dois estágios de Ackerberg, Caves e Fraser (2015). O método é baseado em modelo estrutural de Olley e Pakes (1996) e discussões/avanços longos de Levinsohn e Petrin (2003) e Wooldridge (2009). O método é resumidamente apresentado abaixo – abstraindo da discussão teórica e da estratégia de identificação.

A estratégia de estimação é baseada em dois estágios. No primeiro estágio é computado um modelo semiparamétrico para identificar um termo de erro que ajuda na correta identificação dos parâmetros $\beta_k$ e $\beta_l$ em (1). No segundo estágio os parâmetros $\beta_k$ e $\beta_l$ são estimados por GMM.
No primeiro estágio estime por OLS o seguinte modelo:

$y_{jt} = \beta_0 + \beta_k k_{jt} + \beta_l l_{jt} + f^{−1}_t (k_{jt}, l_{jt},m_{jt}) + \varepsilon_{jt}$ (2)
$y_{jt} = \Phi_t(k_{jt}, l_{jt},m_{jt}) + \varepsilon_{jt}$

tal que $\omega_{jt} \equiv f^{−1}_t (k_{jt}, l_{jt},m_{jt})$, tal que m são os insumos intermediários. O termo $f^{−1}_t (k_{jt}, l_{jt},m_{jt})$ é um polinômio de quarta ordem. O polinômio é o seguinte:

$f^{−1}_t (k_{jt}, l_{jt},m_{jt}) = \sum_{i=0}^4 \beta_{k,l,i,4−i}k^i_{jt} l^{4−i}_{jt} + \sum_{i=0}^4 \beta_{k,m,i,4−i}k^i_{jt} m^{4−i}_{jt} + \sum_{i=0}^4 \beta_{m,l,i,4−i}m^i_{jt} l^{4−i}_{jt} $ (3)

Após estimar (2) calcule e salve o valor predito de $\Phi_t(·)$:

$ \hat{\Phi_t}(k_{jt}, l_{jt},m_{jt}) $(4)

Assumindo que a produtividade tem dinâmica AR(1), $\omega_{jt} = \rho \omega_{jt−1} + \xi_{jt}$, o
segundo estágio é formado pelos momentos
$E[(\xi_{jt} + \varepsilon_{jt}).Z_{jt}] = 0$

tal que $Z_{jt}$ é o vetor de instrumentos. Substituindo $\xi_{jt}$, $\varepsilon_{jt}$ e (4) na equação de momentos, temos:

$E[(y_{jt} - \beta_0 - \beta_k k_{jt} - \beta_l l_{jt} -\rho(\hat{\Phi_{t-1}}(k_{jt-1}, l_{jt-1},m_{jt-1}) - \beta_{0} - \beta_k k_{jt-1} - \beta_l l_{jt-1}))  \times \begin{pmatrix} 
  1 \\ 
  k_{jt} \\
  l_{jt-1} \\
  \hat{\Phi_{t-1}}(k_{jt-1}, l_{jt-1},m_{jt-1})
  \end{pmatrix}]$  (5)

O termo $\hat{\rho}$ pode ser estimado usando a estrutura AR(1) da produtividade $\omega_{jt} = \rho \omega_{jt−1} + \xi_{jt}$. Isto é, o momento necessário para estimar $\hat{\rho}$ não é incluido no sistema GMM. Portanto, para cada loop do algoritmo de estimação é possível estimar o $\hat{\rho}$ com base na produtividade.
Para reduzir o problema, estime $\beta_0$ no primeiro estágio, $\tilde{\beta_0}$, e substitua em (5). Alternativamente, se for conveniente assuma um valor fixo para $\hat{\rho}$, por
exemplo 0.9528. Dado isso, o modelo a ser estimado simplificado para

$E[(y_{jt} - \tilde{\beta_0} - \beta_k k_{jt} - \beta_l l_{jt} -\hat{\rho}(\hat{\Phi_{t-1}}(k_{jt-1}, l_{jt-1},m_{jt-1})- \tilde{beta_0} - \beta_k k_{jt-1} - \beta_l l_{jt-1}))  \times \begin{pmatrix} 
  k_{jt} \\
  l_{jt-1} \\
\end{pmatrix}]$ (6)

Observe que este modelo é exatamente identificado.


Base de dados

A base é de firmas chilenas de 1996 a 2006. As variáveis são:

• Y : ln do produto deflacionado, medido como valor adicionado

• sX : ln do estoque de capital deflacionado (k)

• fX1 : ln do emprego 1 (blue c)

• fX2 : ln do emprego 2 (white c)

• pX : ln de materiais (m)

• cX : ln do investimento deflacionado

• inv = cX

• idvar : id firma

• timevar : id tempo (ano)

A medida de emprego a ser usada deve ser a soma de fX1 e fX2.

1. Estime os parâmetros $\beta_k$, $\beta_l$ por GMM utilizando o estimador apresentado acima. Escreva um programa em Matlab utilizando o solver fminsearch.

In [1]:
import pandas as pd

chilean = pd.read_csv('chilean.csv')


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

df = chilean.rename(columns={'Y': 'y', 'sX': 'k', 'pX': 'm', 'fX1': 'fx1', 'fX2': 'fx2'}).copy()


# l = ln(exp(fx1) + exp(fx2))
df['l'] = np.log(np.exp(df['fx1']) + np.exp(df['fx2']))

# Selecionar variáveis relevantes e criar uma cópia explícita
df_panel = df[['idvar', 'timevar', 'y', 'k', 'l', 'm']].copy()

# Ordenar dados para lagging
df_panel.sort_values(['idvar', 'timevar'], inplace=True)

# Primeiro Estágio: OLS

# Criar termos polinomiais de 4ª ordem para f^-1(k, l, m)
X_stage1 = pd.DataFrame(index=df_panel.index)
X_stage1['const'] = 1
X_stage1['k'] = df_panel['k']
X_stage1['l'] = df_panel['l']

# $\sum_{i=0}^4 \beta k^i * l^{4-i}$
X_stage1['l_4'] = df_panel['l']**4
X_stage1['k_l_3'] = df_panel['k'] * (df_panel['l']**3)
X_stage1['k2_l2'] = (df_panel['k']**2) * (df_panel['l']**2)
X_stage1['k3_l'] = (df_panel['k']**3) * df_panel['l']
X_stage1['k_4'] = df_panel['k']**4

# m^4 + k*m^3 + k^2*m^2 + k^3*m (k^4 já foi definido)
X_stage1['m_4'] = df_panel['m']**4
X_stage1['k_m_3'] = df_panel['k'] * (df_panel['m']**3)
X_stage1['k2_m2'] = (df_panel['k']**2) * (df_panel['m']**2)
X_stage1['k3_m'] = (df_panel['k']**3) * df_panel['m']

# m*l^3 + m^2*l^2 + m^3*l (l^4 e m^4 já foram definidos)
X_stage1['m_l_3'] = df_panel['m'] * (df_panel['l']**3)
X_stage1['m2_l2'] = (df_panel['m']**2) * (df_panel['l']**2)
X_stage1['m3_l'] = (df_panel['m']**3) * df_panel['l']

y_stage1 = df_panel['y']


data_stage1_full = pd.concat([y_stage1, X_stage1], axis=1)
data_stage1_clean = data_stage1_full.dropna()

y_stage1_clean = data_stage1_clean['y']
X_stage1_clean = data_stage1_clean.drop(columns=['y'])

model_stage1 = sm.OLS(y_stage1_clean, X_stage1_clean)
results_stage1 = model_stage1.fit()
print(results_stage1.summary())

beta_0_tilde = results_stage1.params['const']
print(f"\n Intercepto do primeiro estágio: {beta_0_tilde}")

# Calcular Phi_hat e adicionar de volta ao df_panel original usando o índice
df_panel['Phi_hat'] = np.nan
df_panel.loc[X_stage1_clean.index, 'Phi_hat'] = results_stage1.predict(X_stage1_clean)



                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.824
Model:                            OLS   Adj. R-squared:                  0.823
Method:                 Least Squares   F-statistic:                     843.9
Date:                Wed, 18 Jun 2025   Prob (F-statistic):               0.00
Time:                        13:33:12   Log-Likelihood:                -2369.4
No. Observations:                2544   AIC:                             4769.
Df Residuals:                    2529   BIC:                             4856.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.2094      0.140     65.592      0.0

In [ ]:
# Segundo Estágio: GMM

# Criar variáveis defasadas
df_panel['k_lag1'] = df_panel.groupby('idvar')['k'].shift(1)
df_panel['l_lag1'] = df_panel.groupby('idvar')['l'].shift(1)
df_panel['Phi_hat_lag1'] = df_panel.groupby('idvar')['Phi_hat'].shift(1)

# Dados para o segundo estágio (remover NaNs de defasagens)
# Selecionar apenas as colunas necessárias para o GMM
df_stage2 = df_panel[['y', 'k', 'l', 'Phi_hat_lag1', 'k_lag1', 'l_lag1']].dropna()

y_jt = df_stage2['y'].values
k_jt = df_stage2['k'].values
l_jt = df_stage2['l'].values
Phi_hat_t_minus_1 = df_stage2['Phi_hat_lag1'].values
k_jt_minus_1 = df_stage2['k_lag1'].values
l_jt_minus_1 = df_stage2['l_lag1'].values # Usado para produtividade defasada e como instrumento

rho_hat = 0.9528 # Valor fixo conforme a questão

# Função de momentos para GMM
def gmm_moments(params_gmm):
    beta_k, beta_l = params_gmm
    
    # Resíduo u_jt da Equação (6)
    # u_jt = y_jt - beta_0_tilde - beta_k*k_jt - beta_l*l_jt - rho_hat * (Phi_hat_{t-1} - beta_0_tilde - beta_k*k_{t-1} - beta_l*l_{t-1})
    
    term_current_prod = y_jt - beta_0_tilde - beta_k * k_jt - beta_l * l_jt
    term_lagged_prod_expectation = Phi_hat_t_minus_1 - beta_0_tilde - beta_k * k_jt_minus_1 - beta_l * l_jt_minus_1
    
    u_jt = term_current_prod - rho_hat * term_lagged_prod_expectation
    
    # Condições de momento E[u_jt * Z_jt] = 0
    # Z_jt = [k_jt, l_jt_minus_1]
    moment1 = np.mean(u_jt * k_jt)           # Instrumento: k_jt
    moment2 = np.mean(u_jt * l_jt_minus_1)   # Instrumento: l_jt-1
    
    return np.array([moment1, moment2])

# Função objetivo para GMM (soma dos quadrados dos momentos)
# Para GMM exatamente identificado, a matriz de ponderação W pode ser a identidade.
def gmm_objective(params_gmm):
    moments = gmm_moments(params_gmm)
    return np.sum(moments**2) # Equivalente a m'I m

# Valores iniciais para beta_k, beta_l (podem vir de um OLS simples ou serem arbitrários razoáveis)
# Usando os coeficientes de k e l do primeiro estágio como ponto de partida pode ser uma boa ideia.
# No entanto, a questão implica que beta_k e beta_l do primeiro estágio são diferentes dos do segundo.
# Vamos usar valores iniciais comuns para elasticidades.
initial_params_gmm = [0.1, 0.6] 

print("\nSegundo Estágio - GMM:")
# Otimização usando Nelder-Mead (análogo ao fminsearch do Matlab)
result_gmm = minimize(gmm_objective, initial_params_gmm, method='Nelder-Mead')

print("Optimization successful.")
beta_k_gmm, beta_l_gmm = result_gmm.x
print(f"   Final GMM objective function value: {result_gmm.fun:.6e}")
print(f"   Estimated beta_k: {beta_k_gmm:.4f}")
print(f"   Estimated beta_l: {beta_l_gmm:.4f}")


2. Calcule a estatística de Wald (F)